In [1]:
import sys
import torch
import time, os
import numpy as np

folder_path = '../models/'
if folder_path not in sys.path:
    sys.path.append(folder_path)

folder_path = '../data/'
if folder_path not in sys.path:
    sys.path.append(folder_path)
    

from cnmp import CNMP

from data_generators import *


torch.set_float32_matmul_precision('high')

def get_free_gpu():
    gpu_util = []
    for i in range(torch.cuda.device_count()):
        torch.cuda.set_device(i)  # Switch GPU
#        gpu_util.append((i, torch.cuda.memory_stats()['reserved_bytes.all.current'] / (1024 ** 2)))
        gpu_util.append((i, torch.cuda.utilization()))
    gpu_util.sort(key=lambda x: x[1])
    return gpu_util[0][0]

if torch.cuda.is_available():
    available_gpu = get_free_gpu()
    if available_gpu == 0:
        device = torch.device("cuda:0")
    else:
        device = torch.device(f"cuda:{available_gpu}")
else:
    device = torch.device("cpu")

print("Device :", device)

Device : cuda:0


In [3]:
dx, dy, dg, dpe = 1, 1, 1, 1
num_demos, num_val = 3, 1
num_trajs = num_demos + num_val
t_steps = 200
n_max, m_max = 10, 10
num_peaks_max = 4

x = torch.linspace(0, 1, t_steps).view(-1, 1)

y, pp = one_peak(num_trajs)

# interp = True
# if interp:
#     val_ids = torch.from_numpy(np.random.choice(np.arange(1, num_trajs-1), num_val, replace=False))
#     train_ids = [i for i in range(num_trajs) if i not in val_ids]
# else:
#     train_ids = torch.arange(num_demos)
#     val_ids = torch.arange(num_demos, num_trajs)
train_ids = torch.tensor([0, 1, 3])
val_ids = torch.tensor([2])

# gamma_train = pp[train_ids].clone()
# gamma_val = pp[val_ids].clone()

test_cond_ind = int(pp[val_ids, 0].item() * 200)

y_train = y[train_ids].clone()
y_val = y[val_ids].clone()

x_train = x.unsqueeze(0).repeat(num_demos, 1, 1)
x_val = x.unsqueeze(0).repeat(num_val, 1, 1)

# Print shapes
# print(f"x_train shape: {x_train.shape}, gamma_train shape: {gamma_train.shape}, y_train shape: {y_train.shape}")
# print(f"x_val shape: {x_val.shape}, gamma_val shape: {gamma_val.shape}, y_val shape: {y_val.shape}")

IndexError: too many indices for tensor of dimension 2

In [ ]:
import matplotlib.pyplot as plt
# plot train trajectories on the left and validation trajectories on the right
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
for i in range(num_demos):
    axs[0].plot(x_train[i, :, 0], y_train[i, :, 0], label=f"Demo {i}")
for i in range(num_val):
    axs[1].plot(x_val[i, :, 0], y_val[i, :, 0], label=f"Val {i}", color='black', linestyle='dashed')
axs[0].set_title("Training")
axs[1].set_title("Test")
axs[0].grid(True)
axs[1].grid(True)
plt.show()

In [ ]:
batch_size = 1

model_ = CNMP(input_dim=dpe, output_dim=dy, n_max=n_max, m_max=m_max, encoder_hidden_dims=[128, 128, 128], decoder_hidden_dims=[128, 128, 128], batch_size=batch_size, device=device)
optimizer = torch.optim.Adam(lr=3e-4, params=model_.parameters())

pytorch_total_params = sum(p.numel() for p in model_.parameters())
print(pytorch_total_params)


if torch.__version__ >= "2.0":
    model = torch.compile(model_)
else:
    model = model_

66818


In [ ]:
dpe_aug = dpe #+ dg  # dg for gamma (peak_positions)

obs = torch.zeros((batch_size, n_max, dpe_aug+dy), dtype=torch.float32, device=device)
tar_x = torch.zeros((batch_size, m_max, dpe_aug), dtype=torch.float32, device=device)
tar_y = torch.zeros((batch_size, m_max, dy), dtype=torch.float32, device=device)
obs_mask = torch.zeros((batch_size, n_max), dtype=torch.bool, device=device)
tar_mask = torch.zeros((batch_size, m_max), dtype=torch.bool, device=device)

def prepare_masked_batch(t: list, traj_ids: list):
    obs.fill_(0)
    tar_x.fill_(0)
    tar_y.fill_(0)
    obs_mask.fill_(False)
    tar_mask.fill_(False)

    for i, traj_id in enumerate(traj_ids):
        traj = t[traj_id]

        # gamma = torch.zeros(dg)
        # for ppid, pp in enumerate(gamma_train[traj_id]):
        #     gamma[ppid] = pp  # gamma: peak_positions

        n = torch.randint(5, n_max, (1,)).item()
        m = torch.randint(1, m_max, (1,)).item()

        permuted_ids = torch.randperm(t_steps)
        n_ids = permuted_ids[:n]
        m_ids = permuted_ids[n:n+m]
        
        obs[i, :n, :dpe] = x_train[traj_id, n_ids]
        # obs[i, :n, dpe:dpe_aug] = gamma
        obs[i, :n, dpe_aug:] = traj[n_ids]  # SM(t)
        obs_mask[i, :n] = True
        
        tar_x[i, :m, :dpe] = x_train[traj_id, m_ids]
        # tar_x[i, :m, dpe:] = gamma
        tar_y[i, :m] = traj[m_ids]
        tar_mask[i, :m] = True

val_obs = torch.zeros((batch_size, n_max, dpe_aug+dy), dtype=torch.float32, device=device)
val_tar_x = torch.zeros((batch_size, t_steps, dpe_aug), dtype=torch.float32, device=device)
val_tar_y = torch.zeros((batch_size, t_steps, dy), dtype=torch.float32, device=device)
val_obs_mask = torch.zeros((batch_size, n_max), dtype=torch.bool, device=device)

def prepare_masked_val_batch(t: list, traj_ids: list, fixed_ind=None):
    val_obs.fill_(0)
    val_tar_x.fill_(0)
    val_tar_y.fill_(0)
    val_obs_mask.fill_(False)

    for i, traj_id in enumerate(traj_ids):
        traj = t[traj_id]
        
        # gamma = torch.zeros(dg)
        # for ppid, pp in enumerate(gamma_val[traj_id]):
        #     gamma[ppid] = pp  # gamma: peak_positions

        n = 1 #torch.randint(5, n_max, (1,)).item()

        permuted_ids = torch.randperm(t_steps)
        n_ids = permuted_ids[:n]
        m_ids = torch.arange(t_steps)
        if fixed_ind != None:
            n_ids[-1] = fixed_ind
        
        val_obs[i, :n, :dpe] = x_train[traj_id, n_ids]
        # val_obs[i, :n, dpe:dpe_aug] = gamma
        val_obs[i, :n, dpe_aug:] = traj[n_ids]
        val_obs_mask[i, :n] = True
        
        val_tar_x[i, :, :dpe] = x_train[traj_id]
        # val_tar_x[i, :, dpe:] = gamma
        val_tar_y[i] = traj[m_ids]

In [ ]:
import time
import os

timestamp = int(time.time())
root_folder = f'../outputs/tests/simple/cnmp/{str(timestamp)}/'

if not os.path.exists(root_folder):
    os.makedirs(root_folder)

if not os.path.exists(f'{root_folder}saved_models/'):
    os.makedirs(f'{root_folder}saved_models/')

img_folder = f'{root_folder}img/'
if not os.path.exists(img_folder):
    os.makedirs(img_folder)

torch.save(y_train, f'{root_folder}y.pt')

epochs = 2000000
epoch_iter = num_demos // batch_size
v_epoch_iter = num_val//batch_size
avg_loss = 0
val_per_epoch = 1000
min_val_loss = 1000000
mse_loss = torch.nn.MSELoss()

plot_validation = True


for epoch in range(epochs):
    epoch_loss = 0

    traj_ids = torch.randperm(num_demos)[:batch_size * epoch_iter].chunk(epoch_iter)

    for i in range(epoch_iter):
        prepare_masked_batch(y_train, traj_ids[i])

        optimizer.zero_grad()
        
        pred = model(obs, tar_x, obs_mask)

        # if epoch % 10000 == 0 and i == 0:
        #     for k in range(batch_size):
        #         tmp_pred = pred.clone().detach().cpu().numpy()
        #         plt.plot(y_train[k, :, 0].cpu().numpy(), label=f"True {k}")
        #         plt.plot(tmp_pred[k, :, 0], label=f"Pred {k}")
                
        #         plt.legend()
        #         plt.savefig(f'{img_folder}train_{epoch}_{i}_{k}.png')
        #         plt.clf()

        loss = model.loss(pred, tar_y, tar_mask)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if epoch % val_per_epoch == 0 and epoch > 0:
        with torch.no_grad():
            v_traj_ids = torch.randperm(num_val)[:batch_size*v_epoch_iter].chunk(v_epoch_iter)
            val_loss = 0

            for j in range(v_epoch_iter):
                prepare_masked_val_batch(y_val, v_traj_ids[j], test_cond_ind)
                pred = model.val(val_obs, val_tar_x, val_obs_mask)
                if plot_validation:
                    for k in range(batch_size):
                        plt.plot(val_tar_y[k, :, 0].cpu().numpy(), label=f"True {k}")
                        plt.plot(pred[k, :, 0].cpu().numpy(), label=f"Pred {k}")
                        
                        plt.legend()
                        plt.savefig(f'{img_folder}test_{epoch}_{j}_{k}.png')
                        plt.clf()
                val_loss += mse_loss(pred[:, :, :model.output_dim], val_tar_y).item()
                
            if val_loss < min_val_loss:
                min_val_loss = val_loss
                print(f'New best: {min_val_loss}')
                torch.save(model_.state_dict(), f'{root_folder}saved_models/pemp.pt')


    if epoch % 10000 == 0:
        with torch.no_grad():
            prepare_masked_val_batch(y_train, [0], 50)
            pred = model.val(val_obs, val_tar_x, val_obs_mask)
            if plot_validation:
                for k in range(batch_size):
                    plt.plot(val_tar_y[k, :, 0].cpu().numpy(), label=f"True {k}")
                    plt.plot(pred[k, :, 0].cpu().numpy(), label=f"Pred {k}")
                    
                    plt.legend()
                    plt.savefig(f'{img_folder}train_{epoch}_0_{k}.png')
                    plt.clf()

    avg_loss += epoch_loss

    if epoch % 100 == 0:
        print("Epoch: {}, Loss: {}".format(epoch, avg_loss / 100))
        avg_loss = 0

Epoch: 0, Loss: 0.013821499347686768
Epoch: 100, Loss: -3.241593602895737
Epoch: 200, Loss: -3.824243447581539
Epoch: 300, Loss: -3.5602921224944293
Epoch: 400, Loss: -3.952747987918556
Epoch: 500, Loss: -3.8908502727746965
Epoch: 600, Loss: -4.014199901521206
Epoch: 700, Loss: -4.130079568773508
Epoch: 800, Loss: -4.159585261121392
Epoch: 900, Loss: -3.7550580152869224
New best: 0.003498917678371072
Epoch: 1000, Loss: -4.270809175471368
Epoch: 1100, Loss: -4.214950807280839
Epoch: 1200, Loss: -4.135398717038333
Epoch: 1300, Loss: -3.9627201600931583
Epoch: 1400, Loss: -4.103485914245248
Epoch: 1500, Loss: -4.041332601904869
Epoch: 1600, Loss: -4.084904345273972
Epoch: 1700, Loss: -3.679541718065739
Epoch: 1800, Loss: -4.0549556664377455
Epoch: 1900, Loss: -4.089748479276896
Epoch: 2000, Loss: -4.007343905568123
Epoch: 2100, Loss: -4.139815073683858
Epoch: 2200, Loss: -4.224575989693403
Epoch: 2300, Loss: -4.075187349915504
Epoch: 2400, Loss: -4.200768035203218
Epoch: 2500, Loss: -4.26

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>